# Planner Worker Solver

# Setting Up
Uncomment to install the package

In [111]:
# pip install -U "langgraph>=0.2,<0.3" "langchain>=0.3,<0.4" "langchain-openai>=0.2,<0.3" "tavily-python"

Uncomment if API key is not added yet

In [112]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["TAVILY_API_KEY"] = getpass.getpass()

# Building a Planner Worker Solver chatbot
* Planner, uses LLM call
* Worker, sometimes uses LLM call (only when tool name is LLM)
* Solver, uses LLM call

In [114]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph, START, MessagesState

model = ChatOpenAI(model="gpt-4o")

graph_builder = StateGraph(MessagesState)

## Build Planner

In [115]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List


class PlanDetail(BaseModel):
    plan: str = Field(description="This is the plan, a specific step or approach designed to solve a problem in a structured and sequential manner.")
    tool: str = Field(description="This is the tool, an external resource or function that an AI model (like an LLM) can use to retrieve information or perform specific tasks during a problem-solving process.")
    input: str = Field(description="This is the input, the information or data that is provided to the tool to perform its specific task.")
    step_name: str = Field(description="This is the step name, a unique identifier or label for each plan step.")

class PlannerResponse(BaseModel):
    plan_details: List[PlanDetail] = Field(description="This is a list of PlanDetail objects, each containing a plan and its corresponding tool.")


llm_with_structured_output = model.with_structured_output(PlannerResponse)

PLANNER_PROMPT = """
For the following task, make plans that can solve the problem step by step. For each plan, indicate
which external tool together with tool input. Each plan should be a specific step or approach designed

Tools can be one of the following:
(1) Google: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
task: Analyze the economic impact of tourism on New York City over the past decade and compare it to that of Los Angeles.

plan: Use Google to collect data on the economic impact of tourism on New York City over the past decade.
tool: Google
input: "Economic impact of tourism on New York City over the past decade"
step_name: "step#1"

plan: Use Google to collect data on the economic impact of tourism on Los Angeles over the past decade.
tool: Google
input: "Economic impact of tourism on Los Angeles over the past decade"
step_name: "step#2"

plan: Use LLM to compare the data from step#1 and step#2.
tool: LLM
input: "Compare the economic impact of tourism on New York City and Los Angeles over the past decade using step#1 and step#2."
step_name: "step#3"

plan: Use LLM to analyze the factors contributing to any differences or similarities found.
tool: LLM
input: "Analyze the factors that contribute to the differences or similarities in the economic impact of tourism between New York City and Los Angeles based on step#3."
step_name: "step#4"

plan: Use LLM to summarize the overall analysis.
tool: LLM
input: "Summarize the key findings from step#3 and step#4 into a coherent analysis comparing the economic impact of tourism on New York City and Los Angeles over the past decade."
step_name: "step#5"
"""

planner_template = ChatPromptTemplate.from_messages([
    ("system", PLANNER_PROMPT),
    ("user", "{query}")
])

planner_llm = planner_template | model

user_input = "What is the hometown of the 2023 U.S. Open (Tennis) men winner"

response_context = planner_llm.invoke({"query": user_input})
response = llm_with_structured_output.invoke(response_context.content)
plan_details = response.plan_details
print(plan_details)

[PlanDetail(plan="Use Google to find out the winner of the 2023 U.S. Open (Tennis) men's singles.", tool='Google', input="2023 U.S. Open Tennis men's singles winner", step_name='step#1'), PlanDetail(plan='Use Google to find the hometown of the winner identified in step#1.', tool='Google', input="hometown of [winner's name]", step_name='step#2')]


## Build Worker

In [116]:
import os
from langchain_community.tools import TavilySearchResults
from tavily import TavilyClient

tavily_search_results_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True
)
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
WORKER_PROMPT = """
You are a helpful assistant that can perform tasks based on instructions provided by the user.
Provide clear and concise responses to the user's queries. Do not ask for additional information.
"""
worker_template = ChatPromptTemplate.from_messages([
    ("system", WORKER_PROMPT),
    ("user", "{query}")
])

worker_llm = worker_template | model

def worker_perform_plan(plan_detail: PlanDetail) -> str:
    task = plan_detail.plan
    tool = plan_detail.tool
    input_query = plan_detail.input
    step_name = plan_detail.step_name
    result = ""

    if tool == "Google":
        # Perform Google search using Tavily
        response = tavily_client.search(input_query)
        raw_result = response["results"]
        # join result from title and content
        result = "\n".join([r["title"] + "\n" + r["content"] for r in raw_result])
    elif tool == "LLM":
        # Perform LLM task
        response = worker_llm.invoke({"query": input_query})
        result = response.content
    else:
        raise ValueError(f"Invalid tool specified: {tool}")

    return result

evidences = []
for plan_detail in plan_details:
    evidences.append(worker_perform_plan(plan_detail))
print(evidences)

['2023 US Open - Men\'s singles - Wikipedia\nDjokovic became the oldest US Open men\'s singles champion in the Open Era, at 36 years and 111 days, as well as the first man to capture the Australian Open, the French Open, and the US Open in a season since Mats Wilander in 1988.[1] Contents\n2023 US Open – Men\'s singles\nNovak Djokovic defeated Daniil Medvedev in the final, 6–3, 7–6(7–5), 6–3 to win the men\'s singles tennis title at the 2023 US Open. Alcaraz\'s loss marked the 15th consecutive year where the reigning US Open champion failed to defend the title, with Federer being the last man to do so in 2008.\n Other entry information[edit]\nWild cards[edit]\nProtected ranking[edit]\nQualifiers[edit]\nLucky losers[edit]\nWithdrawals[edit]\nReferences[edit]\nExternal links[edit] By reaching a 47th men\'s singles major semifinal, Djokovic surpassed Roger Federer\'s all-time record,[2] and, by reaching the final, he equaled Federer\'s record of reaching all major finals in a season three

## Build Solver

In [117]:
SOLVER_PROMPT = """Using the step-by-step plan and the corresponding evidence provided, solve the following task. Be cautious, as the evidence may be lengthy and could include irrelevant information.

Plans and Evidences:

{plans_and_evidences}

Now, using the provided evidence, solve the task below. Provide only the final answer.
"""

solver_template = ChatPromptTemplate.from_messages([
    ("system", SOLVER_PROMPT),
    ("user", "{query}")
])

solver_llm = solver_template | model

plans_and_evidences_list = []
for plan_detail, evidence in zip(plan_details, evidences):
    plans_and_evidences_list.append(f"Plan: {plan_detail.plan}\nEvidence: {evidence}")
plans_and_evidences = "\n\n".join(plans_and_evidences_list)

response = solver_llm.invoke({"plans_and_evidences": plans_and_evidences, "query": user_input})
print(response.content)

Belgrade, Serbia
